In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import timeit
import random
from itertools import groupby, product, permutations
from scipy import optimize, stats
import shapely
from shapely.geometry import Polygon, Point
import pickle
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda
import torch.nn.functional as F
import torch.nn.init
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable

%matplotlib inline

dbfolder = "G:\\My Drive\\PalankerLab\\retinal modeling\\datasets\\landoltc\\"

In [2]:
# convert times among three systems: real, mea, wn

def timeConversion(t, source = "mea", target = "wn", WNType = "N"):
  
  meaRepRate = 20000
  if WNType == "N":
    stimRepRate = 1 / (33.27082098251457/1000)
  else:
    stimRepRate = 1 / (49.90624667553192/1000)
  
  if source == "mea":
    t = np.divide(t, meaRepRate)
  elif source == "wn":
    t = np.divide(t, stimRepRate)
  elif source == "ms":
    t = np.divide(t, 1000)
  
  if target == "mea":
    t = np.multiply(t, meaRepRate)
  elif target == "wn":
    t = np.multiply(t, stimRepRate)
  elif target == "ms":
    t = np.multiply(t, 1000)
  
  return t

# =====================
# Lists in dictionary to Numpy arrays

def dictLists2npArray(d):
  for k, v in d.items():
    if isinstance(v, dict):
      dictLists2npArray(v)
    elif isinstance(v, list):
      d[k] = np.array(v)

# =====================
# Make geometries

def makeHexagon(l, s = 1):
  a1 = [s * np.cos(np.pi/6), s * np.sin(np.pi/6)]
  corners = [np.array(l)]
  corners.append(l + np.array([a1[0], -a1[1]]))
  corners.append(l + np.array([2 * a1[0], 0]))
  corners.append(l + np.array([2 * a1[0], s]))
  corners.append(l + np.array([a1[0], s + a1[1]]))
  corners.append(l + np.array([0, s]))
  return corners

def makeRectangle(l = (0,0), dim = (1,1)):
  corners = [np.array(l)]
  corners.append(l + np.array([dim[0], 0]))
  corners.append(l + np.array([dim[0], dim[1]]))
  corners.append(l + np.array([0, dim[1]]))
  return corners

def makeBar(center, width, angle):
  R = np.array([[np.cos(angle), -np.sin(angle)], 
                [np.sin(angle), np.cos(angle)]])
  center = np.array(center)
  corners = np.array([[-40, -width/2],
             [-40, width/2],
             [40, width/2],
             [40, -width/2]])
  newCorners = np.matmul(R,corners.T)
  newCorners = newCorners.T + center
  return newCorners

def makeLandoltC(center, diameter, angle):
  R = np.array([[np.cos(angle), -np.sin(angle)], 
                [np.sin(angle), np.cos(angle)]])
  center = np.array(center)
  outer = Polygon(Point(*center).buffer(diameter/2))
  inner = Polygon(Point(*center).buffer (diameter/2 - diameter/5))
  ring = outer.difference(inner)
  opening = np.array([[0, -diameter/10],
                     [diameter/2, -diameter/10],
                     [diameter/2, diameter/10],
                     [0, diameter/10]])
  opening = Polygon(np.matmul(R,opening.T).T + center)
  landoltC = ring.difference(ring.intersection(opening))
  return landoltC
  

# =====================
# Make bar for hexgonal lattice

def buildBarStimulus(params, visualize = False):

  center = params['center']
  width = params['width']
  angle = params['angle']
  startFrame = params['startFrame']
  dims = params['dims']

  origin = [0, 0]
  hexCoord = makeHexagon(origin, 1)
  hx = Polygon(hexCoord)

  xlist = np.arange(dims[1])
  ylist = np.arange(dims[2])

  xylist = list(product(xlist, ylist))
  hx_dict = dict()
  hx_coord = []

  bar = Polygon(makeBar(center, width, angle))
  inter = dict()

  for x, y in xylist:
    xx = np.cos(np.pi/6) * (2*x+y-2*np.floor(y/2))
    yy = 1.5 * y
    hx_coord.append([xx, yy])
    hx_dict[(x,y)] = Polygon(makeHexagon([xx,yy], 1))
    inter[(x,y)] = hx_dict[(x,y)].intersection(bar).area / hx_dict[(x,y)].area #/ 2 + 0.5

  outStim = np.zeros((dims[0],dims[1],dims[2])) #* 0.5
  for x in range(dims[1]):
    for y in range(dims[2]):
      outStim[startFrame:, x, y] = inter[(x,y)]

  if visualize == True:
    plt.subplot(1,2,1)

    for (x,y), hx in hx_dict.items():
      xl, yl = hx.exterior.xy
      plt.plot(xl,yl)
      plt.scatter(*hx.centroid.xy, alpha = inter[(x,y)])

    bx, by = bar.exterior.xy
    plt.plot(bx, by)
    plt.xlim(-1,35)
    plt.ylim(-1,31)

    plt.subplot(1,2,2)

    plt.imshow(outStim[-1,:,:].T)
    plt.show()
  return outStim

# =====================
# Make bar for square lattice

def buildBarStimulus2(params, visualize = False):

  center = params['center']
  width = params['width']
  angle = params['angle']
  startFrame = params['startFrame']
  dims = params['dims']

  origin = [0, 0]
  sqCoord = makeRectangle()
  sq = Polygon(sqCoord)

  xlist = np.arange(dims[1])
  ylist = np.arange(dims[2])

  xylist = list(product(xlist, ylist))
  sq_dict = dict()
  sq_coord = []

  bar = Polygon(makeBar(center, width, angle))
  inter = dict()

  for x, y in xylist:
    sq_coord.append([x, y])
    sq_dict[(x,y)] = Polygon(makeRectangle([x,y]))
    inter[(x,y)] = sq_dict[(x,y)].intersection(bar).area / sq_dict[(x,y)].area #/ 2 + 0.5

  outStim = np.ones((dims[0],dims[1],dims[2])) #* 0.5
  for x in range(dims[1]):
    for y in range(dims[2]):
      outStim[startFrame:, x, y] = inter[(x,y)]

  if visualize == True:
    plt.subplot(1,2,1)

    for (x,y), sq in sq_dict.items():
      xl, yl = sq.exterior.xy
      plt.plot(xl,yl)
      plt.scatter(*sq.centroid.xy, alpha = inter[(x,y)])

    bx, by = bar.exterior.xy
    plt.plot(bx, by)
    plt.xlim(-1,65)
    plt.ylim(-1,33)

    plt.subplot(1,2,2)

    plt.imshow(outStim[-1,:,:].T)
    plt.show()
  return outStim


# =====================
# Make Landolt C on hexgonal lattice

def buildLandoltCStimulus(params, visualize = False):

  center = params['center']
  width = params['width']
  angle = params['angle']
  startFrame = params['startFrame']
  dims = params['dims']

  origin = [0, 0]
  #hexCoord = makeHexagon(origin, 1)
  #hx = Polygon(hexCoord)

  xlist = np.arange(dims[1])
  ylist = np.arange(dims[2])

  xylist = list(product(xlist, ylist))
  c_dict = dict()
  c_coord = []

  print(f"{center} {width} {angle}")
  C = makeLandoltC(center, width, angle)
  inter = dict()

  for x, y in xylist:
    xx = np.cos(np.pi/6) * (2*x+y-2*np.floor(y/2))
    yy = 1.5 * y
    c_coord.append([xx, yy])
    c_dict[(x,y)] = Polygon(makeHexagon([xx,yy], 1))
    inter[(x,y)] = c_dict[(x,y)].intersection(C).area / c_dict[(x,y)].area #/ 2 + 0.5

  outStim = np.zeros((dims[0],dims[1],dims[2])) #* 0.5
  for x in range(dims[1]):
    for y in range(dims[2]):
      outStim[startFrame:, x, y] = inter[(x,y)]

  if visualize == True:
    plt.subplot(1,2,1)

    for (x,y), c in c_dict.items():
      xl, yl = c.exterior.xy
      plt.plot(xl,yl)
      plt.scatter(*c.centroid.xy, alpha = inter[(x,y)])

    bx, by = C.exterior.xy
    plt.plot(bx, by)
    plt.xlim(-1,35)
    plt.ylim(-1,31)

    plt.subplot(1,2,2)

    plt.imshow(outStim[-1,:,:].T)
    plt.show()
  return outStim

# =====================
# Make Landolt C on hexgonal lattice

def buildLandoltCStimulus_natural(params, visualize = False):

  center = params['center']
  width = params['width']
  angle = params['angle']
  startFrame = params['startFrame']
  dims = params['dims']

  origin = [0, 0]

  xlist = np.arange(dims[1])
  ylist = np.arange(dims[2])

  xylist = list(product(xlist, ylist))
  c_dict = dict()
  c_coord = []

  print(f"{center} {width} {angle}")
  C = makeLandoltC(center, width, angle)
  
  outStim = np.zeros((dims[0],dims[1],dims[2])) #* 0.5

  for x, y in xylist:
    c_dict[(x,y)] = Polygon(makeRectangle([x,y], (1, 1)))
    outStim[startFrame:, x, y] = c_dict[(x,y)].intersection(C).area / c_dict[(x,y)].area #/ 2 + 0.5

  
#   for x in range(dims[1]):
#     for y in range(dims[2]):
#       outStim[startFrame:, x, y] = inter[(x,y)]

  if visualize == True:
    plt.subplot(1,2,1)

    for (x,y), c in c_dict.items():
      xl, yl = c.exterior.xy
      plt.plot(xl,yl)
      plt.scatter(*c.centroid.xy, alpha = outStim[-1,x,y])

    bx, by = C.exterior.xy
    plt.plot(bx, by)
    plt.xlim(-1,35)
    plt.ylim(-1,31)

    plt.subplot(1,2,2)

    plt.imshow(outStim[-1,:,:].T)
    plt.show()
  return outStim

In [5]:
dsfolder = "G:\\My Drive\\PalankerLab\\retinal modeling\\datasets\\"

cellfile = 'retina_LE_V.json'
#cellfile = 'retina_LE_E.json'
#cellfile = 'retina_RCS_E.json'
#filename = 'RCS_spikeTimes_cell391.json'
#filename = 'LE_E_spikeTimes_cell301.json'
with open(dsfolder + cellfile,"r") as datafile:
  cell = json.load(datafile)

# counter = 0
# cell = dict()
# for key, val in cell1.items():
#     counter += 1
#     if key == '5311' or key == '5566': cell[key] = val

#print(cell)
dictLists2npArray(cell)

In [10]:
#  === EDIT THIS BLOCK ===
n_output = 2
lum = (0.5, 1)
tol = 0.1
horizBar = (7, 2) #(start, width)
vertBar = (7, 2)
startFrame = 13
# === ===

# 2. 
plotDuration = timeConversion(2, source = 'real', target = 'mea', WNType = 'N')
binWidth = timeConversion(0.01, source = 'real', target = 'mea', WNType = 'N')
nbins = int(plotDuration / binWidth)

popRaster = dict()
spikingPattern = dict()
spikingPattern['info'] = dict()
spikingPattern['info']['stimulusType'] = 'crossbar'

cx = np.arange(10) + 13
cy = np.arange(10) + 10
#cx = [30]
#cy = [12]
centerList = [[x, y] for x in cx for y in cy]

widthList = 2 * np.arange(8) + 4
#widthList = [4]
angleList = np.arange(4)

# 3.
stims = dict()

WNType = cell[random.choice(list(cell.keys()))]['experiment']['info']['WNType']

if WNType == 'N': dims = (24,64,32)
else: dims = (24,20,20)

params = dict()
outStim = dict()
for center in centerList:
    for width in widthList:
        for angle in angleList:
            key = ((int(center[0]),int(center[1])), int(width), int(angle))
            params[key] = dict()
            params[key]['center'] = center
            params[key]['width'] = width
            params[key]['angle'] = angle * np.pi/2
            params[key]['startFrame'] = 20
            params[key]['dims'] = dims
            
            if np.random.random() > 1:
                outStim[key] = buildLandoltCStimulus_natural(params[key], visualize = True)
            else:
                outStim[key] = buildLandoltCStimulus_natural(params[key], visualize = False)


[13, 10] 4 0.0
[13, 10] 4 1.5707963267948966
[13, 10] 4 3.141592653589793
[13, 10] 4 4.71238898038469
[13, 10] 6 0.0
[13, 10] 6 1.5707963267948966
[13, 10] 6 3.141592653589793
[13, 10] 6 4.71238898038469
[13, 10] 8 0.0
[13, 10] 8 1.5707963267948966
[13, 10] 8 3.141592653589793
[13, 10] 8 4.71238898038469
[13, 10] 10 0.0
[13, 10] 10 1.5707963267948966
[13, 10] 10 3.141592653589793
[13, 10] 10 4.71238898038469
[13, 10] 12 0.0
[13, 10] 12 1.5707963267948966
[13, 10] 12 3.141592653589793
[13, 10] 12 4.71238898038469
[13, 10] 14 0.0
[13, 10] 14 1.5707963267948966
[13, 10] 14 3.141592653589793
[13, 10] 14 4.71238898038469
[13, 10] 16 0.0
[13, 10] 16 1.5707963267948966
[13, 10] 16 3.141592653589793
[13, 10] 16 4.71238898038469
[13, 10] 18 0.0
[13, 10] 18 1.5707963267948966
[13, 10] 18 3.141592653589793
[13, 10] 18 4.71238898038469
[13, 11] 4 0.0
[13, 11] 4 1.5707963267948966
[13, 11] 4 3.141592653589793
[13, 11] 4 4.71238898038469
[13, 11] 6 0.0
[13, 11] 6 1.5707963267948966
[13, 11] 6 3.1415

KeyboardInterrupt: 

In [ ]:
cx = np.arange(10) + 5
cy = np.arange(10) + 5

centerList = [(x, y) for x in cx for y in cy]

In [36]:
lc_map = dict()
for key, val in params.items():
    lc_map[str(key)] = dict()
    lc_map[str(key)]['params'] = dict()
    lc_map[str(key)]['params']['center'] = [int(i) for i in params[key]['center']]
    lc_map[str(key)]['params']['width'] = int(params[key]['width'])
    lc_map[str(key)]['params']['angle'] = int(params[key]['angle'] / np.pi * 2)
    lc_map[str(key)]['params']['dims'] = params[key]['dims']
    lc_map[str(key)]['stimulus'] = outStim[key].astype(float).tolist()

In [29]:
lc_map['((5, 5), 15, 3)']

{'params': {'center': [5, 5], 'width': 15, 'angle': 3, 'dims': (24, 20, 20)},
 'stimulus': [[[0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   [0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0],
   [0

In [37]:
outfile = "landoltC_stimulus_dict.json"
fn = dbfolder + outfile
print(fn)
with open(fn, 'w') as fout:
    json.dump(lc_map, fout)

G:\My Drive\PalankerLab\retinal modeling\datasets\landoltc\landoltC_stimulus_dict.json
